___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>


# Manually Creating a DQN Model


## Deep-Q-Learning
In this notebook we will create our first Deep Reeinforcement Learning model, called Deep-Q-Network (DQN)
We are again using a simple environment from openai gym. <br />
However, you will soon see the enormous gain we will get by switching from standard Q-Learning to Deep Q Learning.

In this notebook we again take a look at the CartPole problem (https://gym.openai.com/envs/CartPole-v1/)



Let us start by importing the necessary packages

# Part 0: Imports

Notice how we're importing the TF libraries here at the top together, in some rare instances, if you import them later on, you get strange bugs, so best just to import everything from Tensorflow here at the top.

In [1]:
from collections import deque
import random

import numpy as np
import gym  # Contains the game we want to play
from tensorflow.keras.models import Sequential  # To compose multiple Layers
from tensorflow.keras.layers import Dense  # Fully-Connected layer
from tensorflow.keras.layers import Activation  # Activation functions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import clone_model

# Part 1: The Environment

In [2]:
env_name = 'CartPole-v1'
env = gym.make(env_name)  # create the environment

Remember, the goal of the CartPole challenge was to balance the stick upright

In [3]:
env.reset()  # reset the environment to the initial state
for _ in range(200):  # play for max 200 iterations
    env.render(mode="human")  # render the current game state on your screen
    random_action = env.action_space.sample()  # chose a random action
    env.step(random_action)  # execute that action
env.close()  # close the environment

C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\gym\envs\classic_control\cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


# Part 2: The Artificial Neural Network

### Let us build our first Neural Network
To build our network, we first need to find out how many actions and observation our environment has.
We can either get those information from the source code (https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py) or via the following commands:

In [4]:
num_actions = env.action_space.n
num_observations = env.observation_space.shape[0]  # You can use this command to get the number of observations
print(f"There are {num_actions} possible actions and {num_observations} observations")


There are 2 possible actions and 4 observations


So our network needs to have an input dimension of 4 and an output dimension of 2.
In between we are free to chose.

Let's just say we want to use a four layer architecture:


1. The first layer has 16 neurons
2. The second layer has 32 neurons
4. The fourth layer (output layer) has 2 neurons

This yields 690 parameters
$$ \text{4 observations} * 16 (\text{neurons}) + 16 (\text{bias}) + (16*32) + 32 + (32*2)+2 = 690$$

In [5]:
model = Sequential()

model.add(Dense(16, input_shape=(1, num_observations)))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))


model.add(Dense(num_actions))
model.add(Activation('linear'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 activation (Activation)     (None, 1, 16)             0         
                                                                 
 dense_1 (Dense)             (None, 1, 32)             544       
                                                                 
 activation_1 (Activation)   (None, 1, 32)             0         
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
 activation_2 (Activation)   (None, 1, 2)              0         
                                                                 
Total params: 690
Trainable params: 690
Non-trainable pa

Now we have our model which takes an observation as input and outputs a value for each action.
The higher the value, the more likely that this value is a suitable action for the current observation

As stated in the lecture, Deep-Q-Learning works better when using a target network.
So let's just copy the above network

In [6]:
#model.load_weights("34.ckt")
target_model = clone_model(model)


Now it is time to define our hyperparameters.

# Part 3: Hyperparameters and Update Function

In [7]:
EPOCHS = 1000

epsilon = 1.0
EPSILON_REDUCE = 0.995  # is multiplied with epsilon each epoch to reduce it
LEARNING_RATE = 0.001 #NOT THE SAME AS ALPHA FROM Q-LEARNING FROM BEFORE!!
GAMMA = 0.95


Let us use the epsilon greedy action selection function once again:

In [8]:
def epsilon_greedy_action_selection(model, epsilon, observation):
    if np.random.random() > epsilon:
        prediction = model.predict(observation)  # perform the prediction on the observation
        action = np.argmax(prediction)  # Chose the action with the higher value
    else:
        action = np.random.randint(0, env.action_space.n)  # Else use random action
    return action

As shown in the lecture, we need a replay buffer.
We can use the **deque** data structure for this, which already implements the circular behavior.

The *maxlen* argument specifies the number of elements the buffer can store between he overwrites them at the beginning

The following cell shows an example usage of the deque function. You can see, that in the first example all values fit into the deque, so nothing is overwritten. 

In the second example, the deque is printed in each iteration. It can hold all values in the first five iterations but then needs to delete the oldest value in the deque to make room for the new value 

In [9]:
### deque examples
deque_1 = deque(maxlen=5)
for i in range(5):  # all values fit into the deque, no overwriting
    deque_1.append(i)
print(deque_1)
print("---------------------")
deque_2 = deque(maxlen=5)

# after the first 5 values are stored, it needs to overwrite the oldest value to store the new one
for i in range(10):  
    deque_2.append(i)
    print(deque_2)

deque([0, 1, 2, 3, 4], maxlen=5)
---------------------
deque([0], maxlen=5)
deque([0, 1], maxlen=5)
deque([0, 1, 2], maxlen=5)
deque([0, 1, 2, 3], maxlen=5)
deque([0, 1, 2, 3, 4], maxlen=5)
deque([1, 2, 3, 4, 5], maxlen=5)
deque([2, 3, 4, 5, 6], maxlen=5)
deque([3, 4, 5, 6, 7], maxlen=5)
deque([4, 5, 6, 7, 8], maxlen=5)
deque([5, 6, 7, 8, 9], maxlen=5)


Let's say we allow our replay buffer a maximum size of 20000

In [10]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

As mentioned in the lecture, action replaying is crucial for Deep Q-Learning. <br />
The following cell implements one version of the action replay algorithm. <br />
It uses the zip statement paired with the * (Unpacking Argument Lists) operator to create batches from the samples for efficient prediction and training.<br />
The zip statement returns all corresponding pairs from each entry. <br />
It might look confusing but the following example should clarify it

In [11]:
test_tuple = [(1, 2, 3), (4, 5, 6), (7, 8, 9)]
zipped_list = list(zip(*test_tuple))
a, b, c = zipped_list
print(a, b, c)

(1, 4, 7) (2, 5, 8) (3, 6, 9)


Now it's time to write the replay function

In [12]:
def replay(replay_buffer, batch_size, model, target_model):
    
    # As long as the buffer has not enough elements we do nothing
    if len(replay_buffer) < batch_size: 
        return
    
    # Take a random sample from the buffer with size batch_size
    samples = random.sample(replay_buffer, batch_size)  
    
    # to store the targets predicted by the target network for training
    target_batch = []  
    
    # Efficient way to handle the sample by using the zip functionality
    zipped_samples = list(zip(*samples))  
    states, actions, rewards, new_states, dones = zipped_samples  
    
    # Predict targets for all states from the sample
    targets = target_model.predict(np.array(states))
    
    # Predict Q-Values for all new states from the sample
    q_values = model.predict(np.array(new_states))  
    
    # Now we loop over all predicted values to compute the actual targets
    for i in range(batch_size):  
        
        # Take the maximum Q-Value for each sample
        q_value = max(q_values[i][0])  
        
        # Store the ith target in order to update it according to the formula
        target = targets[i].copy()  
        if dones[i]:
            target[0][actions[i]] = rewards[i]
        else:
            target[0][actions[i]] = rewards[i] + q_value * GAMMA
        target_batch.append(target)

    # Fit the model based on the states and the updated targets for 1 epoch
    model.fit(np.array(states), np.array(target_batch), epochs=1, verbose=0)  


We need to update our target network every once in a while. <br />
Keras provides the *set_weights()* and *get_weights()* methods which do the work for us, so we only need to check whether we hit an update epoch

In [13]:
def update_model_handler(epoch, update_target_model, model, target_model):
    if epoch > 0 and epoch % update_target_model == 0:
        target_model.set_weights(model.get_weights())


# Part 4: Training the Model

Now it is time to write the training loop! <br />
First we compile the model

In [14]:
model.compile(loss='mse', optimizer=Adam(lr=LEARNING_RATE))


C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Then we perform the training routine. <br />
This might take some time, so make sure to grab your favorite beverage and watch your model learn. <br />
Feel free to use our provided chekpoints as a starting point

In [15]:
best_so_far = 0
for epoch in range(EPOCHS):
    observation = env.reset()  # Get inital state
    
    # Keras expects the input to be of shape [1, X] thus we have to reshape
    observation = observation.reshape([1, 4])  
    done = False  
    
    points = 0
    while not done:  # as long current run is active
        
        # Select action acc. to strategy
        action = epsilon_greedy_action_selection(model, epsilon, observation)
        
        # Perform action and get next state
        next_observation, reward, done, info = env.step(action)  
        next_observation = next_observation.reshape([1, 4])  # Reshape!!
        replay_buffer.append((observation, action, reward, next_observation, done))  # Update the replay buffer
        observation = next_observation  # update the observation
        points+=1

        # Most important step! Training the model by replaying
        replay(replay_buffer, 32, model, target_model)

    
    epsilon *= EPSILON_REDUCE  # Reduce epsilon
    
    # Check if we need to update the target model
    update_model_handler(epoch, update_target_model, model, target_model)
    
    if points > best_so_far:
        best_so_far = points
    if epoch %25 == 0:
        print(f"{epoch}: Points reached: {points} - epsilon: {epsilon} - Best: {best_so_far}")


0: Points reached: 21 - epsilon: 0.995 - Best: 21


ValueError: in user code:

    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\salimibeni\anaconda3\envs\gymTF\lib\site-packages\keras\engine\input_

# Part 5: Using Trained Model

In [ ]:
observation = env.reset()
for counter in range(300):
    env.render()
    
    # TODO: Get discretized observation
    action = np.argmax(model.predict(observation.reshape([1,4])))
    
    # TODO: Perform the action 
    observation, reward, done, info = env.step(action) # Finally perform the action
    
    if done:
        print(f"done")
        break
env.close()